In [ ]:
!pip install pandas pymongo
!pip install python-dotenv



In [ ]:
from google.colab import files

uploaded = files.upload()


Saving interactions_train.csv to interactions_train.csv
Saving interactions_validation.csv to interactions_validation.csv
Saving ingr_map.pkl to ingr_map.pkl
Saving interactions_test.csv to interactions_test.csv
Saving RAW_recipes.csv to RAW_recipes.csv
Saving RAW_interactions.csv to RAW_interactions.csv
Saving PP_recipes.csv to PP_recipes.csv
Saving PP_users.csv to PP_users.csv


In [ ]:
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd

# Cargar los archivos CSV en DataFrames
recipes = pd.read_csv('RAW_recipes.csv')
interactions_train = pd.read_csv('interactions_train.csv')
interactions_test = pd.read_csv('interactions_test.csv')
interactions_validation = pd.read_csv('interactions_validation.csv')


In [ ]:
print("Recipes:")
print(recipes.head())  # Muestra las primeras 5 filas

print("\nInteractions Train:")
print(interactions_train.head())

print("\nInteractions Test:")
print(interactions_test.head())

print("\nInteractions Validation:")
print(interactions_validation.head())


Recipes:
                                         name      id  minutes  \
0  arriba   baked winter squash mexican style  137739       55   
1            a bit different  breakfast pizza   31490       30   
2                   all in the kitchen  chili  112140      130   
3                          alouette  potatoes   59389       45   
4          amish  tomato ketchup  for canning   44061      190   

   contributor_id   submitted  \
0           47892  2005-09-16   
1           26278  2002-06-17   
2          196586  2005-02-25   
3           68585  2003-04-14   
4           41706  2002-10-25   

                                                tags  \
0  ['60-minutes-or-less', 'time-to-make', 'course...   
1  ['30-minutes-or-less', 'time-to-make', 'course...   
2  ['time-to-make', 'course', 'preparation', 'mai...   
3  ['60-minutes-or-less', 'time-to-make', 'course...   
4  ['weeknight', 'time-to-make', 'course', 'main-...   

                                    nutrition  n_steps  \


In [ ]:
import pandas as pd

# Cargar los datos
recipes = pd.read_csv('RAW_recipes.csv')
interactions_train = pd.read_csv('interactions_train.csv')

# 1. Verificar duplicados
print("Duplicados en Recipes:", recipes.duplicated().sum())
print("Duplicados en Interactions Train:", interactions_train.duplicated().sum())

# 2. Eliminar duplicados
recipes = recipes.drop_duplicates()
interactions_train = interactions_train.drop_duplicates()

# 3. Manejar valores faltantes
print("Valores faltantes en Recipes:", recipes.isnull().sum())
print("Valores faltantes en Interactions Train:", interactions_train.isnull().sum())

# 4. Llenar o eliminar valores faltantes según sea necesario
# Por ejemplo, eliminar filas con valores faltantes en la columna 'name'
recipes = recipes.dropna(subset=['name'])

# 5. Revisar los tipos de datos
print(recipes.dtypes)
print(interactions_train.dtypes)

# 6. Convertir tipos de datos si es necesario
# Ejemplo: convertir 'date' a tipo datetime
interactions_train['date'] = pd.to_datetime(interactions_train['date'])


Duplicados en Recipes: 0
Duplicados en Interactions Train: 0
Valores faltantes en Recipes: name                 1
id                   0
minutes              0
contributor_id       0
submitted            0
tags                 0
nutrition            0
n_steps              0
steps                0
description       4979
ingredients          0
n_ingredients        0
dtype: int64
Valores faltantes en Interactions Train: user_id      0
recipe_id    0
date         0
rating       0
u            0
i            0
dtype: int64
name              object
id                 int64
minutes            int64
contributor_id     int64
submitted         object
tags              object
nutrition         object
n_steps            int64
steps             object
description       object
ingredients       object
n_ingredients      int64
dtype: object
user_id        int64
recipe_id      int64
date          object
rating       float64
u              int64
i              int64
dtype: object


In [ ]:
recipes['submitted'] = pd.to_datetime(recipes['submitted'], errors='coerce')
interactions_train['date'] = pd.to_datetime(interactions_train['date'], errors='coerce')


In [ ]:
import pandas as pd

# Supongamos que ya tienes cargados los DataFrames de tus recetas e interacciones
recipes = pd.read_csv('RAW_recipes.csv')
interactions_train = pd.read_csv('interactions_train.csv')

# Prepara los datos de las recetas
recipes_dict = recipes.to_dict(orient='records')  # Convierte a lista de diccionarios

# Prepara los datos de las interacciones
interactions_dict = interactions_train.to_dict(orient='records')  # Convierte a lista de diccionarios


In [ ]:
from pymongo import MongoClient
import os
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()
mongodb_uri = os.getenv('MONGODB_URI')

# Conectar a la base de datos
client = MongoClient(mongodb_uri)
db = client['mi_base_de_datos']  # Nombre de la base de datos
recipes_collection = db['recipes']  # Nombre de la colección para recetas


In [ ]:
import pandas as pd

# Cargar los datos
recipes = pd.read_csv('RAW_recipes.csv')
interactions_train = pd.read_csv('interactions_train.csv')

# Ver los primeros registros de cada DataFrame
print("Recetas:")
print(recipes.head())
print("\nInteracciones:")
print(interactions_train.head())

# Limpiar los datos de recetas
# Manejar valores faltantes
recipes['description'].fillna('', inplace=True)  # Reemplazar valores faltantes en 'description' por una cadena vacía

# Seleccionar columnas relevantes
recipes_cleaned = recipes[['id', 'name', 'description', 'ingredients', 'minutes', 'n_steps', 'steps']]

# Renombrar las columnas para la base de datos
recipes_cleaned.rename(columns={'id': 'recipe_id', 'name': 'name', 'description': 'description',
                                 'ingredients': 'ingredients', 'minutes': 'minutes',
                                 'n_steps': 'n_steps', 'steps': 'steps'}, inplace=True)

# Limpiar los datos de interacciones
# Seleccionar columnas relevantes
interactions_cleaned = interactions_train[['user_id', 'recipe_id', 'date', 'rating']]

# Guardar los DataFrames limpios en archivos CSV
recipes_cleaned.to_csv('cleaned_recipes.csv', index=False)
interactions_cleaned.to_csv('cleaned_interactions.csv', index=False)

print("Datos limpios y guardados en archivos CSV.")


Recetas:
                                         name      id  minutes  \
0  arriba   baked winter squash mexican style  137739       55   
1            a bit different  breakfast pizza   31490       30   
2                   all in the kitchen  chili  112140      130   
3                          alouette  potatoes   59389       45   
4          amish  tomato ketchup  for canning   44061      190   

   contributor_id   submitted  \
0           47892  2005-09-16   
1           26278  2002-06-17   
2          196586  2005-02-25   
3           68585  2003-04-14   
4           41706  2002-10-25   

                                                tags  \
0  ['60-minutes-or-less', 'time-to-make', 'course...   
1  ['30-minutes-or-less', 'time-to-make', 'course...   
2  ['time-to-make', 'course', 'preparation', 'mai...   
3  ['60-minutes-or-less', 'time-to-make', 'course...   
4  ['weeknight', 'time-to-make', 'course', 'main-...   

                                    nutrition  n_steps  \


<ipython-input-19-3b2d634ace8b>:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  recipes['description'].fillna('', inplace=True)  # Reemplazar valores faltantes en 'description' por una cadena vacía
<ipython-input-19-3b2d634ace8b>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recipes_cleaned.rename(columns={'id': 'recipe_id', 'name': 'nam

Datos limpios y guardados en archivos CSV.


In [ ]:
from google.colab import files

# Descargar los archivos CSV
files.download('cleaned_recipes.csv')
files.download('cleaned_interactions.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install pymongo
!pip install dnspython


In [ ]:
from pymongo import MongoClient

# URI de conexión a MongoDB (modifica según sea necesario)
MONGODB_URI = 'mongodb://localhost:27017/mi_base_de_datos'

# Crear una conexión al cliente de MongoDB
client = MongoClient(MONGODB_URI)

# Seleccionar la base de datos
db = client['mi_base_de_datos']

# Seleccionar la colección de recetas
recipes_collection = db['recipes']

# Seleccionar la colección de interacciones
interactions_collection = db['interactions']

# Seleccionar la colección de Users
users_collection = db['Users']

# Prueba de conexión
print("Conexión exitosa a MongoDB")


Conexión exitosa a MongoDB
